In [1]:
import os
import torch
from torch import nn    
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


# Define the Class

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='mps:0')


# Model Layers

In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0690,  0.1035, -0.0637,  0.3539, -0.9859,  0.0708, -0.5470, -0.0826,
         -0.9082,  0.2725,  0.0770, -0.2746, -0.5952,  0.4109, -0.1039,  0.2605,
          0.8416, -0.1899,  0.7332, -0.0673],
        [ 0.2197,  0.0737,  0.0756,  0.3011, -0.6798,  0.1529, -0.2758,  0.1392,
         -0.8399, -0.0419,  0.0973, -0.4716, -0.3898,  0.5717,  0.1737,  0.1576,
          0.5160, -0.1353,  0.1927, -0.2710],
        [-0.1687,  0.0726,  0.3782,  0.2286, -0.6083,  0.0857, -0.5599,  0.3825,
         -0.5653,  0.3609,  0.4463, -0.5876, -0.3112,  0.4518, -0.0022,  0.2945,
          0.5001, -0.0807,  0.4723,  0.0238]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0690, 0.1035, 0.0000, 0.3539, 0.0000, 0.0708, 0.0000, 0.0000, 0.0000,
         0.2725, 0.0770, 0.0000, 0.0000, 0.4109, 0.0000, 0.2605, 0.8416, 0.0000,
         0.7332, 0.0000],
        [0.2197, 0.0737, 0.0756, 0.3011, 0.0000, 0.1529, 0.0000, 0.1392, 0.0000,
         0.0000, 0.0973, 0.0000, 0.0000, 0.5717, 0.17

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model Parameters

In [17]:
print("Model structure:", {model}, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: {NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)} 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0070,  0.0114,  0.0157,  ...,  0.0238, -0.0354, -0.0354],
        [ 0.0143, -0.0185,  0.0095,  ...,  0.0040,  0.0090,  0.0334]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0236, -0.0073], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0400,  0.0305, -0.0150,  ...,  0.0021, -0.0417, -0.0275],
        [-0.0076, -0.0326, -0.0347,  ...,  0.0043, -0.0079, -0.0287]],
       device='mps:0', grad_fn=<Sl